## ...

In [1]:
version = '7'

In [2]:
import xlrd
import os, sys
import re
import calendar
from glob import glob
import yaml

from functools import partial
from itertools import cycle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# plt.style.use('_classic_test')
# plt.style.use('seaborn-paper')
# plt.style.use('seaborn-pastel')
# plt.style.use('ggplot')


In [3]:
with open('configurations_v{}.yaml'.format(version), 'r') as out_stream:
    CONFIG = yaml.load(out_stream, Loader=yaml.FullLoader)

In [4]:
temp_dataset_path = CONFIG['observations']['temperature']['dataset_path']
temp_output_path = CONFIG['observations']['temperature']['output_path']
prec_dataset_path = CONFIG['observations']['percepitation']['dataset_path']
prec_output_path = CONFIG['observations']['percepitation']['output_path']
temp_obs_col = CONFIG['observations']['temperature']['column']

cordex_dataset_path = CONFIG['models']['cordex']['dataset_path']
cordex_output_path = CONFIG['models']['cordex']['output_path']
cordex_start_year = CONFIG['models']['cordex']['time']['start']['year']
cordex_hour_interval = CONFIG['models']['cordex']['time']['hour_interval']
cordex_start_timestamp = CONFIG['models']['cordex']['time']['timestamp']['start']
cordex_end_timestamp = CONFIG['models']['cordex']['time']['timestamp']['end']
cordex_freq_timestamp = CONFIG['models']['cordex']['time']['timestamp']['freq']
cordex_temp_filename = CONFIG['models']['cordex']['variables']['temperature']['filename']

surfex_dataset_path = CONFIG['models']['surfex']['dataset_path']
surfex_output_path = CONFIG['models']['surfex']['output_path']
surfex_start_year = CONFIG['models']['surfex']['time']['start']['year']
surfex_hour_interval = CONFIG['models']['surfex']['time']['hour_interval']
surfex_start_timestamp = CONFIG['models']['surfex']['time']['timestamp']['start']
surfex_end_timestamp = CONFIG['models']['surfex']['time']['timestamp']['end']
surfex_freq_timestamp = CONFIG['models']['surfex']['time']['timestamp']['freq']
surfex_temp_filename = CONFIG['models']['surfex']['variables']['temperature']['filename']

cordex_dict_raw = CONFIG['models']['cordex']['raw_names']
surfex_dict_raw = CONFIG['models']['surfex']['raw_names']
models_cols = CONFIG['models']['columns']
models_markers = CONFIG['plots']['names_markers']

plots_output_path = CONFIG['plots']['output']
seasons_dict = CONFIG['dates']['seasons_dict']
period = CONFIG['period']['historical']
metrics_fn = CONFIG['metrics']['func']
metrics_symbols = CONFIG['metrics']['symbols']

ens_alpha_col = CONFIG['ensembles']['alpha']['column']
ens_beta_col = CONFIG['ensembles']['beta']['column']


METRIC_NAMES = metrics_fn.keys()
SEASONS = seasons_dict.keys()
METRIC_NAMES, SEASONS

(dict_keys(['bias', 'sigma', 'perkins', 'yk']),
 dict_keys(['DJF', 'MAM', 'JJA', 'SON']))

In [5]:
list(zip(['temp_dataset_path',
'temp_output_path',
'temp_obs_col',
'prec_dataset_path',
'prec_output_path',
'cordex_dataset_path',
'cordex_output_path',
'cordex_start_year',
'cordex_hour_interval',
'cordex_start_timestamp',
'cordex_end_timestamp',
'cordex_freq_timestamp',
'cordex_temp_filename',
'surfex_dataset_path',
'surfex_output_path',
'surfex_start_year',
'surfex_hour_interval',
'surfex_start_timestamp',
'surfex_end_timestamp',
'surfex_freq_timestamp',
'surfex_temp_filename',
'cordex_dict_raw',
'surfex_dict_raw',
'models_markers',
'plots_output_path',
'seasons_dict',
'period',
'metrics_fn',
'metrics_symbols',
'ens_alpha_col',
'ens_beta_col',
'models_cols'],
[temp_dataset_path,
temp_output_path,
temp_obs_col,
prec_dataset_path,
prec_output_path,
cordex_dataset_path,
cordex_output_path,
cordex_start_year,
cordex_hour_interval,
cordex_start_timestamp,
cordex_end_timestamp,
cordex_freq_timestamp,
cordex_temp_filename,
surfex_dataset_path,
surfex_output_path,
surfex_start_year,
surfex_hour_interval,
surfex_start_timestamp,
surfex_end_timestamp,
surfex_freq_timestamp,
surfex_temp_filename,
cordex_dict_raw,
surfex_dict_raw,
models_markers,
plots_output_path,
seasons_dict,
period,
metrics_fn,
metrics_symbols,
ens_alpha_col,
ens_beta_col,
models_cols
]))

[('temp_dataset_path', '..\\data\\observations\\temp'),
 ('temp_output_path', '..\\data\\observations\\temp\\output'),
 ('temp_obs_col', 17),
 ('prec_dataset_path', '..\\data\\observations\\prec'),
 ('prec_output_path', '..\\data\\observations\\prec\\output'),
 ('cordex_dataset_path', '..\\data\\cordex'),
 ('cordex_output_path', '..\\data\\cordex\\output'),
 ('cordex_start_year', 1971),
 ('cordex_hour_interval', 3),
 ('cordex_start_timestamp', '01-01-1971 03'),
 ('cordex_end_timestamp', '31-12-2005 21'),
 ('cordex_freq_timestamp', '3H'),
 ('cordex_temp_filename', 'Forc2m_TA.txt'),
 ('surfex_dataset_path', '..\\data\\surfex'),
 ('surfex_output_path', '..\\data\\surfex\\output'),
 ('surfex_start_year', 1971),
 ('surfex_hour_interval', 3),
 ('surfex_start_timestamp', '01-01-1971 03'),
 ('surfex_end_timestamp', '31-12-2005 21'),
 ('surfex_freq_timestamp', '3H'),
 ('surfex_temp_filename', 'Hourly_T2m.txt'),
 ('cordex_dict_raw',
  {0: 'CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CNRM-ALADIN63_v2

### Write cordex/surfex models dict to YAML

In [6]:
# cordex_dict_raw = {i:name for i,name in enumerate(select_models_folders(cordex_dataset_path)[0])}
# surfex_dict_raw = {i:name for i,name in enumerate(select_models_folders(surfex_dataset_path)[0])}

# models_markers = {
#     0: ('CNRMCM5_ALADIN', '*'),
#     1: ('CNRMCM5_HIRHAM', '*'),
#     2: ('CNRMCM5_SMHIRCA', '*'),
#     3: ('ECEARTH_SMHIRCA', '+'),
#     4: ('ECEARTH_HIRHAM', '+'),
#     5: ('IPSLCM5-SMHIRCA', 'd'),
#     6: ('HADGEM_REGCM', '1'),
#     7: ('HADGEM_SMHIRCA', '1'),
#     8: ('MPIESM_COSMO', 'o'),
#     9: ('MPIESM_HIRHAM', 'o'),
#     10: ('MPIESM_REGCM', 'o'),
#     11: ('MPIESM_SMHIRCA', 'o'),
#     12: ('MPIESM_REMO', 'o'),
#     13: ('NORESM_COSMO', '^'),
#     14: ('NORESM_HIRHAM', '^'),
#     15: ('NORESM_REMO', '^'),
#     16: ('NORESM_SMHIRCA', '^')}

# metrics_symbols = {'bias': '|Bias|', 'sigma': r'|$\sigma_n^{-1}$|',
#                 'perkins': '|S|', 's': '|S|', 'yk': '|YK|'}

# with open('configurations_v7.yaml', 'a') as in_stream:
#     yaml.dump(metrics_symbols, in_stream)
# #     yaml.dump(models_markers, in_stream)

## Read observations

In [7]:
# OPTIMIZE!
# REDO


def get_xls_files(variable, obs_path):
    '''
        variable: 'temp' or 'prec'
        glob_path: path to file type excel, used by glob
            example: '..\\data\\observations\\*.xls'
            
        Returns a dict of all file_paths in the following format
            file_path': (variable, year)
            item example: '..\\data\\observations\\PREC1979.XLS': ('prec', '1979')

    '''
    glob_path = os.path.join(os.getcwd(), obs_path) + '\*.xls'    # the '\'  in '\*.xls' is nedded
    print(glob_path)
    
    def all_4digits_year(dict_files):
        '''
            returns the dict of files with porper year yyyy
        '''
        return {file:(pair[0], '19'+ pair[1] if len(pair[1]) == 2 else pair[1]) for file,pair in dict_files.items()}

    file_pattern = re.compile(rf'{variable}\s*(?P<year>\d*).xls' , flags=re.IGNORECASE)
    files = {f:(variable, re.search(file_pattern, os.path.basename(f)).group(1)) for f in glob(glob_path) if re.match(file_pattern, os.path.basename(f))}
    return all_4digits_year(files)


def get_data_from_xls(filename_path, year):
    '''
    
    
    '''
    xls = pd.ExcelFile(filename_path)
    df_raw = pd.read_excel(xls, header=None, usecols=range(0,25), index=False)
    # clean rows with no data (first col empty, must have a day or a month)
    df_raw.dropna(axis=0, how='all', subset=[0], inplace=True)

    # df to hold final result
    columns = ['month', 'day', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    df_temp = pd.DataFrame(columns = columns)

    months_list = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez', 'stop']
    months_dict = {m:i for i,m in enumerate(months_list[:-1], 1)}
    def month_iter(months):     # allows to consume all the months once, it uses a 'stop' word to easing the parser
        for month in months:
            yield month

    # regex for 1-31
    day_pattern = re.compile(r'(?P<day>([1-9]|[12]\d|3[01]))$') 

    # month iterator (can be done with a simple list as well)
    month_it = month_iter(months_list)
    month = next(month_it) # starts with 'jan'

    for row in df_raw.iterrows():
        label = row[1][0]                                                  # for each row the first col
        month_pattern = re.compile(rf'{month}' , flags=re.IGNORECASE)
        if re.match(month_pattern, str(label)):                      # assumes that the first line has the 'jan' matching pattern
            prev_month = month
            try:
                month = next(month_it)
            except:
                break                                               # the end of the list
        if re.match(day_pattern, str(label)):
            day = label                                             # it is a day
            new_series = pd.Series([months_dict[prev_month], day]).append(row[1][1:]) # select only the values (temp/prec)
            new_series.index = df_temp.columns
            df_temp = df_temp.append(new_series, ignore_index=True)
    df_temp['year'] = year      
    return df_temp[['year'] + columns]



def get_all_obs_data(dict_files, variable, out_dir = None):
    '''
        Returns a df with all temperature series
        Numbers are rounded to 2 decimal places
        Saves the df to  ..\\data\\observations\\output\\
    '''
    # order file paths by year, ascending.
    files_years = sorted([(path,year[1]) for path,year in dict_files.items()], key = lambda v: v[1])

    columns = ['year', 'month', 'day', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    df_data = pd.DataFrame(columns = columns)

    for file_path,year in files_years:
        df_temp = get_data_from_xls(file_path, year)
        assert (df_temp.shape[0] <= 366) and (df_temp.shape[0] >= 365), '365 or 366 days in file'
        print('file_path - {}\tyear - {}\tlen: {}'.format(file_path, year, df_temp.shape[0]))
        df_data = df_data.append(df_temp)

    # fix dtypes    
    dict_types = {col:'float' for col in df_data.columns}
    dict_types['year'], dict_types['month'], dict_types['day']  = 'int', 'int', 'int'
    df_data = df_data.astype(dict_types)
    # round to 2 decimal places
    df_data = df_data.round(decimals = {col:2 for col in columns[3:]})
    
    
    
    
    # save dataframe to csv
    filename = '{}_{}_{}.csv'.format(variable, files_years[0][1], files_years[-1][1])
    save_df2csv(df_data, filename, out_dir)
    return df_data.reset_index(drop=True)



## Read Models

In [8]:
def select_models_folders(dir_models = None, period = period):
    folders = [folder for folder in  os.listdir(os.path.join(os.getcwd(), dir_models)) if period in folder]
    return folders, len(folders)

In [9]:
def add_season2df(df):
    cols = df.columns
    conditions = [(df['month'].isin([12, 1, 2])), (df['month'].isin([3, 4, 5])),
                (df['month'].isin([6, 7, 8])), (df['month'].isin([9, 10, 11]))]
    df['season'] = np.select(conditions, seasons_dict.keys())
    cols = cols.insert(1, 'season')
    return df[cols]

In [10]:
def get_data_models(dir_models = None, 
                    var_filename = None, 
                    start_timestamp = cordex_start_timestamp,
                    end_timestamp = cordex_end_timestamp,
                    freq_timestamp = cordex_freq_timestamp, plot = False):
    '''
        For temperature only!! Convertion to C
        
    '''
    res = []                        # to accumulate the dataseries
    color = cycle('rbgkmc')
    folders, nmodels = select_models_folders(dir_models, period)
    for model_id, folder in enumerate(folders):
        T2m = np.loadtxt(os.path.join(dir_models, folder, var_filename)) - 273.15
        T2m = pd.Series(T2m, name = model_id, index=pd.date_range(
                                                    start = start_timestamp,
                                                    end = end_timestamp,
                                                    freq = freq_timestamp,
                                                    dayfirst=True ))
        # replace -9999 for np.nan
        if T2m[0] < -100: T2m[0] = np.nan
            
        if plot:
            T2m.plot(figsize=(18,8), color=next(color))
            plt.title('Model: {}'.format(model_id))
            plt.show();
        res.append(T2m) 
        
    res = pd.concat(res, axis = 1)
    res.index.name = 'date'
    
    if plot:
        res.plot(figsize=(18,8))
        plt.tight_layout()
        plt.show();
    return res

## Save and load prepared data

In [11]:
def save_datafile(save_fn, filename, output_dir):
    '''
        out_filename: defines what is computed, ex. temp_sea_avg_obs
    '''
    out_dir = os.path.join(os.getcwd(), output_dir)
    if not os.path.exists(out_dir):
        print('Output folder does not exist. Created a new one: {}'.format(out_dir))
        os.makedirs(out_dir)
    if filename[-4:] != '.csv':
        filename = filename + '.csv'
    save_fn(path_or_buf = os.path.join(os.getcwd(), out_dir, filename))


def save_df2csv(df, filename, output_dir, index=False):
    '''
        
    '''
    # to get other args when called
    save_fn = partial(df.to_csv, sep=',', header=True, index=index, date_format='%Y-%m-%d %H:%M:%S')
    return save_datafile(save_fn, filename, output_dir)




def save_plot(plot, filename, output_dir = plots_output_path):
    '''
        out_filename: defines what is computed, ex. temp_sea_avg_obs
    '''
    out_dir = os.path.join(os.getcwd(), output_dir)
    if not os.path.exists(out_dir):
        print('Output folder does not exist. Created a new one: {}'.format(out_dir))
        os.makedirs(out_dir)
    path_or_buf = os.path.join(os.getcwd(), out_dir, filename)
    plot.savefig(path_or_buf,
                 dpi=None,
                 facecolor='w',
                 edgecolor='w',
                 orientation='portrait',
                 papertype=None, format=None,
                 transparent=False,
                 bbox_inches=None,
                 pad_inches=0.1,
                 frameon=None,
                 metadata=None)
    

In [12]:
def load_csv2df(filename, filepath, freq_index=None):
    '''
        if it is a timeseries 'date' and a 'freq_index' value is passed it is the index
    '''
    def conv_to_int(c):    # to guarantee that the hours' column names are type int
        try:
            c = int(c)
        except:
            return c
        return c
    
    filename_path = os.path.join(os.getcwd(), filepath, filename)
    date_dtype = {'year':np.int32, 'month':np.int32, 'day':np.int32}
    df = pd.read_csv(filename_path, sep=',', parse_dates=True, dtype = date_dtype)
    
    df.columns = [conv_to_int(col) for col in df.columns]
    if ('date' in df.columns) and (freq_index):
        df.set_index('date', inplace=True) # if it is a timeseries 'date' is the index
        df.index = pd.DatetimeIndex(df.index.values, freq=freq_index)
        df.index.name = 'date'
    return df



def compare_saved_loaded_df(df_, df):
    assert df_.index.equals(df.index)
    assert df_.columns.equals(df.columns)
    assert np.isclose(df_.values, df.values, equal_nan=True).all()
    return True

## Pre-processing

In [13]:
def obs_ts_format(df, existing_cols, new_col_name, var_name, freq_index='3H'):
    '''
    
    '''
    df_temp = df.copy(deep=True)
    df_temp = pd.melt(df_temp, id_vars=existing_cols, var_name=new_col_name, value_name=var_name)

    df_temp = add_season2df(df_temp)                  # add season column
    df_temp['hour'] = df_temp['hour'].astype(int)     # 'hour' must be int
    df_temp.reset_index(drop=True, inplace=True)
    
    df_temp['date'] = df_temp.apply(lambda s: pd.datetime(*map(int, (s.year, s.month, s.day, s.hour)))
                                  , axis=1)
    df_temp = df_temp.set_index('date')
    df_temp.drop(['year','month','day','hour'], axis=1, inplace=True)
    df_temp.sort_index(inplace=True)
    df_temp = df_temp.asfreq(freq_index)
    return df_temp

In [14]:
def filter_df_by_season(df, season = None, month_col='month'):
    '''
    
    '''
    df = df.copy()
    return df.loc[df['month'].isin(seasons_dict[season]), :]


def filter_ts_by_hour(df, hour = None):
    '''
    
    '''
    df = df.copy()
    return df[df.index.hour==hour]



def filter_ts_by_season(ts, season = None):
    '''
    
    '''
    ts = ts.copy()
    return ts.loc[ts.index.month.isin(seasons_dict[season]), :]


def df_all_models(list_df_models):
    cols = list_df_models[0].columns
    for cmod,model in enumerate(list_df_models):
        model['model'] = cmod
    df =  pd.concat([*list_df_models], axis=0)
    return df[['model', *cols]]



def remove_duplicates(ts):
    ts = ts.reset_index()
    ts.drop_duplicates(inplace=True)
    ts.set_index('date', inplace=True)
    ts = ts.iloc[:,0]
    return ts

## Processing 

In [15]:
def percentiles_table(df, values, by = 'hour', percentiles = [1,5,25,50,75,95,99]):
    '''
        Returns the percentiles of a value by a specific column (by)
        Organizes a timeseries in tabular form for a specific column
        df header: year	month	day	hour	temp_3h
        
        values: example 'temp_3h'
        
    '''
    index = [col for col in df.columns if col not in [values, by]]
    df  = df.pivot_table(values = values, columns = by, index = index)
    df = df.reset_index(drop=True)
    df_desc = df.describe(percentiles=np.asarray(percentiles)/100)
    df_desc = df_desc.T
    df_desc.reset_index(inplace=True)
    df_desc.rename(columns={'index': by}, inplace=True)
    df_desc.drop(['count', 'std'], axis=1, inplace=True)
    df_desc
    return df_desc.set_index(by)

## Error metrics

In [16]:
def fn_models_obs(func, df_data, model_cols, df_obs):
    '''
        Generic apply function that performs a func between a list of columns (model_cols) and a single column (df_obs)
        
    '''
    return df_data[model_cols].apply(func, obs = df_obs, axis=0)

##### Bias 

In [17]:
def diff_(s, obs):
    # NO NEED to remove duplicates from df_obs - used for Ensemble Beta
    return (obs - s).mean()


def bias(df, model_cols, df_obs):
    return fn_models_obs(diff_, df, model_cols, df_obs), df.mean(axis=0)      # simplify

##### Sigma score - Normalized standard deviation measure

In [18]:
def sigma_(s, obs):
    # accordingly to the sigma definition
    return s.std()/obs.std()


def sigma_score(df, model_cols, df_obs):
    ### Remove duplicates from df_obs -  used for Ensemble Beta
    df_obs = remove_duplicates(df_obs)
    return fn_models_obs(sigma_, df, model_cols, df_obs), df.std()    # simplify

##### S score - Perkins Skill score 

In [19]:
def pdf(series, bins = None):
    '''
        pdf for pandas dataSeries, using a numpy array func
        returns both the density and the bin edges
    '''
    density, _ = np.histogram(series, bins = bins, density=True)
    return density


def  minimun(s, obs):
    return np.minimum(s, obs)

def perkins_skill_score(df, model_cols, df_obs, bins=None):
    cols = model_cols   
    if not bins:
        bins = range(0, 51)   # 0ºC to 50ºC
    # only models, see below for observations (needed for EnsembleBeta, observations remove duplicates)
    models_pdfs = [pdf(s, bins) for _,s in df[cols].items()]
    
    ### Remove duplicates from df_obs  -  used for Ensemble Beta
    df_obs = remove_duplicates(df_obs)
    obs_pdf = pdf(df_obs.values, bins)
    # to export as probe together with models pdfs
    df_obs_pdf = pd.DataFrame(obs_pdf, columns=[temp_obs_col], index=bins[:-1])
    
    # transpose it!    
    df_pdfs = pd.DataFrame(models_pdfs, index=cols, columns=bins[:-1]).T
    # S index calculation
    S = fn_models_obs(minimun, df_pdfs, model_cols, obs_pdf).sum(axis=0)*100
    return S, pd.concat([df_pdfs, df_obs_pdf], axis=1)

##### YK skewness - Yule-Kendall skewness

In [20]:
def YK_old(df, model_cols, df_obs, obs_col=17):
    models_perc = df[model_cols].describe(percentiles=[0.05, 0.5, 0.95])
    ### Remove duplicates from df_obs  -  used for Ensemble Beta
    df_obs = remove_duplicates(df_obs)
    perc_obs = df_obs.describe(percentiles=[0.05, 0.5, 0.95])
    perc_obs.name= obs_col
    df_perc = pd.concat([models_perc, perc_obs], axis=1)
    df_perc = df_perc.iloc[4:7, :]
    return ((df_perc.loc['95%', :] - df_perc.loc['50%', :]) -\
            (df_perc.loc['50%', :] - df_perc.loc['5%', :]))\
            / (df_perc.loc['95%', :] - df_perc.loc['5%', :]), df_perc



def yk_perc(df):
    return ((df.loc['95%', :] - df.loc['50%', :]) -\
            (df.loc['50%', :] - df.loc['5%', :]))\
            / (df.loc['95%', :] - df.loc['5%', :])

def YK_(df, model_cols, df_obs, obs_col=17):
    models_perc = df[model_cols].describe(percentiles=[0.05, 0.5, 0.95])
    ### Remove duplicates from df_obs  -  used for Ensemble Beta
    df_obs = remove_duplicates(df_obs)
    perc_obs = df_obs.describe(percentiles=[0.05, 0.5, 0.95])
    perc_obs.name= obs_col
    df_perc = pd.concat([models_perc, perc_obs], axis=1)
    df_perc = df_perc.iloc[4:7, :]
    return yk_perc(df_perc), df_perc


def yk_diff(s, obs):
    return s-obs

def YK_skewness_by_hour(df, model_cols, obs_col):
    '''
        model_cols: list of columns
    '''
    df_yk, probe = metric_by_hour(YK_, df, model_cols, obs_col)
    return df_yk.loc[model_cols, :].apply(yk_diff, obs = df_yk.loc[obs_col, :], axis=1), probe

In [21]:
def metric_by_hour(metric, df, lst_model_cols, observation_col):
    '''
        with a time series as input, first is filtered for each hour (daily cycle)
        calls metric() to get the metric results, and gathers to a list of results
        
        Returns:
            df with all results organized
                    values: metric
                    cols: hours
                    rows: models
            probes
    '''
    res = []
    probes = []   # to gather intermediate results. Ex: In perkins the models pdfs
    for hour in range(0,22,3):
        df_filtered = filter_ts_by_hour(df, hour)
#         print(df_filtered.shape, df.shape)
        df_obs = df_filtered.loc[:,observation_col]    # separate the observations df
        df_metric, probe = metric(df_filtered, lst_model_cols, df_obs)
#         df_metric, probe = metric(df_filtered.iloc[:,:-1], lst_model_cols, df_obs)
        res.append(df_metric)        
        probes.append(probe)
    return pd.concat(res, axis=1, keys=list(range(0,22,3))), probes

#### pdf, cdf, pmf...

In [22]:
def compute_pdf_cdf(sample, plot=True):
    '''
        Visualize the ECDF, interpolated PDF and samples
        
    '''
    bins=int(np.sqrt(len(sample)))
#     bins =  100
    hist = np.histogram(sample, bins = bins)
    hist_dist = st.rv_histogram(hist)

    sample_min, sample_max = np.min(sample), np.max(sample)
    pdf, Ecdf = hist_dist.pdf, hist_dist.cdf
    assert Ecdf(sample_min) == 0, 'Something wrong with ECDF!'
    assert Ecdf(sample_max) == 1, 'Something wrong with ECDF!'

    if plot:
        X = np.linspace(sample_min, sample_max, bins)
        plt.title("PDF")
        _ = plt.hist(sample, density = True, bins = bins)
        _ = plt.plot(X, pdf(X), label = 'PDF')
        #_ = plt.plot(X, Ecdf(X), label = 'CDF')
        plt.legend()
        plt.margins(0.02)
        plt.show()
    return pdf, Ecdf


def compute_pdf_KDE(sample, bandwidth=2, kernel='gaussian', plot=True):
    '''
    
    '''
    bins=int(np.sqrt(len(sample)))
    #print(bins)
    sample = sample.reshape((len(sample), 1))
    # fit density
    model = KernelDensity(bandwidth=bandwidth, kernel=kernel)
    model.fit(sample)
    
    if plot:
        sample_min, sample_max = np.min(sample), np.max(sample)
        X = np.linspace(sample_min, sample_max, bins)
        values = X
        values = values.reshape((len(values), 1))
        #print(values.shape)
        probabilities = model.score_samples(values)
        probabilities = np.exp(probabilities)
        _ = plt.hist(sample, bins=bins, density=True)
        _ = plt.plot(values[:], probabilities)
        plt.title("pdf using KDE")
        plt.show()
    #return np.exp(model.score_samples)
    return model.score_samples

## Visualizations

In [23]:
def plt_metrics_by_hour(df, title,
                        filename = None,
                        output_dir = plots_output_path,
                        sub_folder=None,
                        df_other=None):
    '''
        Sigma is not Sigma^-1
    '''
    
    
    # fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(15, 30), sharex=True, sharey=True)
    fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(15, 10), sharex=True, sharey=True)  # figsize=(w, h)
    fig.suptitle(title, fontsize=20)
    
    # plot all models and  FIX IT!!!!!!
    n_models2plot = df.index[-1]+1

    
    for hour,ax in zip(range(0,22,3), axs.flat):
        plt.subplot(ax)
        ax.plot(df[hour], '-o')
        if isinstance(df_other, pd.DataFrame):
            plt.plot(df_other[hour], '-o')
        ax.set_title('Hour = {}'.format(hour), fontsize=14)
        plt.grid(axis='x', color='gray', linestyle='--')
        plt.xticks(list(range(n_models2plot)))
        ax.margins(0.05)
    if isinstance(df_other, pd.DataFrame):
        plt.legend(labels=['bias corrected'], loc='best')
    if filename:
        if sub_folder:
            output_dir = os.path.join(output_dir, sub_folder)
        save_plot(plt, filename+'_per_h', output_dir = output_dir)
    plt.show();
    
    df.plot(figsize=(15, 10), style = '-o')
    fig.legend(labels = df.columns, loc='upper right', title='hour')
    plt.xticks(list(range(n_models2plot)))
    plt.margins(0.05)
    plt.grid(axis='x', color='silver', linestyle='--')
    plt.title(title, fontsize=20)
    if filename:
        save_plot(plt, filename, output_dir = output_dir)
    print(output_dir)
    plt.show();

## Ranking

In [25]:
def set_ranks(df_metric, metric=None, scale=None, decimals=None): 
    if metric == 'bias':
        scale = 7
        metric_prep = df_metric.abs().div(scale).round(decimals=1)
        metric_rank = metric_prep.rank(method='dense', ascending=False)
    
    elif metric == 'perkins':
        scale = 100
        metric_prep = df_metric.div(scale).round(decimals=1)
        metric_rank = metric_prep.rank(method='dense', ascending=True)
    
    elif metric == 'sigma':
        scale = 2
        metric_prep = (df_metric - 1).abs().div(scale).round(decimals=1)
        metric_rank = metric_prep.rank(method='dense', ascending=False)
    
    elif metric == 'yk':
        scale = 1
        metric_prep = df_metric.abs().div(scale).round(decimals=1)
        metric_rank = metric_prep.rank(method='dense', ascending=False)
    
    else:
        print('{} is not a valid metric [bias, sigma, perkins, yk]'.format(metric))
    
    print('abs({})\tmin:{}\tmax:{} '.format(metric, df_metric.abs().min().min(), df_metric.abs().max().max()))
    print('transf({})\tmin:{}\tmax:{} '.format(metric, metric_prep.abs().min().min(), metric_prep.abs().max().max()))
    print('{} rank\tmin:{}\tmax:{} '.format(metric, metric_rank.abs().min().min(), metric_rank.abs().max().max()))
    
    return metric_rank


def plot_bump(metric_rank,
              title,
              filename=None,
              output_dir = plots_output_path,
              sub_folder=None, 
              figsize=(18,18)):
    '''
        SIGMA?????
    
    '''
    for row in metric_rank.iterrows():
        delta = row[0]*0.025
        metric_rank.iloc[row[0],:] = row[1] + delta

    metric_rank.T.plot(figsize=(18,18), style='-o')
    plt.yticks(list(range(1, int(metric_rank.max().max())+1)))
    plt.gca().invert_yaxis()
    plt.xticks(list(range(0,22,3)))
    plt.xlabel('hour')
    plt.grid(axis='y')
    plt.margins(0.1)
    plt.title(title)
    plt.legend(loc='lower right');
    if filename:
        if sub_folder:
            output_dir = os.path.join(output_dir, sub_folder)
        save_plot(plt, filename, output_dir = output_dir)

### Models' comparison

In [ ]:
def create_Urb_effect(data):
    '''
        artificialy create data with urban effect
    
    '''
    data = data.copy()
    mu = df_bias_jja.values.reshape(1,-1).mean()
    std = df_bias_jja.values.reshape(1,-1).std()
    return data + std*np.random.randn(*df_bias_jja.shape) + mu



def aux_plot_cmp_metric(metric, ax, hour, data, d_min, d_max, id_letter):#, param_dict):
    """
    A helper function to make a metric comparison graph

    Parameters
    ----------
    ax : Axes. The axes to draw to
    data : Dataframe. The data as a df with 2 cols
    d_min: the floor.min of all data points
    d_max: the ceil.max of all data points
    id_letter: letters to identify the subplots
    #param_dict : dict. Dictionary of kwargs to pass to ax.plot

    Returns
    -------
    out : list
        list of artists added
    """
    metric = metric.lower()
#     metrics_dict = {'bias': '|Bias|', 'sigma': r'|$\sigma_n^{-1}$|',    # TODO - mov to configurations file
#                     'perkins': '|S|', 's': '|S|', 'yk': '|YK|'}
    for idx,row in data.iterrows():
        out = ax.scatter(row[0], row[1], 
                         label=models_markers[idx][0], 
                         marker=models_markers[idx][1],
                         s=100,)
                        # **param_dict)
    if metric in ['bias', 'sigma', 'yk']:
        ax.fill([d_min, d_min, d_max, d_min], [d_min, d_max, d_max, d_min], fill=True, alpha=0.3, color='lightgray')
    elif metric in ['perkins', 's']: # only for perkins
        ax.fill([d_min, d_max, d_max, d_min], [d_min, d_min, d_max, d_min], fill=True, alpha=0.3, color='lightgray')
    else:
        print('Metric not allowed! Chose among: bias, perkins, sigma, yk')
    
    ax.set_xlabel(' '.join(('CORDEX {}'.format(metrics_symbols[metric]), r'T$_{2m}$ $^{o}$C')), fontsize=14)
    ax.set_ylabel(' '.join(('SFX {}'.format(metrics_symbols[metric]), r'T$_{2m}$ $^{o}$C')), fontsize=14)
    
    # outer limits
    dd_min, dd_max= d_min-0.05*d_max, d_max+0.05*d_max
    ax.set_xlim(dd_min, dd_max)
    ax.set_ylim(dd_min, dd_max)
    ax.text(-0.15, 0.95, '{})'.format(next(id_letter)),
            fontsize= 16, transform=ax.transAxes)
    ax.text(0.5, 0.95, r'$hour={}h$'.format(hour),
        horizontalalignment='center', fontsize=14, ha='center', transform=ax.transAxes)
    
    return out




def plot_cmp_metric(metric, data_ref, data_cmp, title_fig=None,
                    out_filename=None, out_dir=plots_output_path, sub_folder=None ):
    '''
        OBS:
            Computed the abs for all metrics: |Bias|,|Perkins|,|YK| and
            |Sigma-1|
    '''
    # fig dimensions
    W, H = 12, 24
    
    
    if metric == 'sigma':
        data_ref = data_ref - 1
        data_cmp = data_cmp - 1
    # ABS
    data_ref, data_cmp = data_ref.abs(), data_cmp.abs()
    
    data_x_min, data_x_max = data_ref.min().min(), data_ref.max().max()
    data_y_min, data_y_max = data_cmp.min().min(), data_cmp.max().max()
    x_min = min([data_x_min, data_x_max, data_y_min, data_y_max])
    x_max = max([data_x_min, data_x_max, data_y_min, data_y_max])
    #x_min = np.floor(x_min)
    if (x_max > 1) and (metric!='sigma'): x_max = np.ceil(x_max)                            # TODO BETTER

    from itertools import cycle
    plot_id = cycle('abcdefghijklmnopqrstuvxwz')  


    hours = range(0,22,3)
    fig, axs = plt.subplots(4,2, figsize=(W, H), frameon=True, constrained_layout=False)
    axs = axs.flat
    for hour, ax in zip(hours, axs):
        data = pd.DataFrame()
        data['CDX'] = data_ref.loc[:,hour]
        data['SFX'] = data_cmp.loc[:,hour]
        plot = aux_plot_cmp_metric(metric, ax,
                               hour,
                               data,
                               x_min, x_max,
                               plot_id,)
                               #param_dict={'zorder':1})

    # get legend form last axe
    handles, labels = ax.get_legend_handles_labels()
    leg = fig.legend(handles, labels, 
                     ncol=3, mode='expand', 
                     frameon=False, loc=8,
                     fontsize=14, markerscale=1)
    fig.suptitle('Season: {}'.format(title_fig), fontsize=14)
#     fig.tight_layout()
    fig.subplots_adjust(top=0.95, bottom=0.1)                         # TRICK
    
    if out_filename:
        if sub_folder:
            out_dir = os.path.join(out_dir, sub_folder)
        save_plot(plt, out_filename, output_dir = out_dir)
        
        
        
        
# Generator for time series comparison
# hourly means
def gen_cdx_sfx_ts_cmp_h_mean(models_seq):
    '''
        This is a generator - yields
        (mean_hourly dataframe , cordex/surfex, season
    '''
    
    
    for mod_sea,path in models_seq:
        filename = '{}_join_obs_{}.csv'.format(mod_sea[0], mod_sea[1].lower())
        df = load_csv2df(filename, path, freq_index='3H')
        # calculate the hourly means
        mean_hourly = df.groupby(df.index.time).mean()
        mean_hourly.index =  list(map(lambda t: t.hour, mean_hourly.index))
        mean_hourly.index.name = 'hour'
        yield mean_hourly,mod_sea[0],mod_sea[1]
        
        
        
def aux_plot_models_obs_hourly_mean(data, ax, title=u'title'):
    for col in (data.loc[:,list(range(17))]):
        ax.plot(data.loc[:, col], alpha=0.2, label=models_markers[col][0])  # linestyle='--'
    ax.plot(data.mean(axis=1), '-o', label='avg')
    out = ax.plot(data.loc[:, 17], '-o', color='red', label='obs')
    ax.set_xticks(list(range(0,22,3)))
    ax.set_title(title)
    ax.set_ylabel(r'T$_{2m}$ $^{o}$C', fontsize=14)
    ax.set_xlabel('hours')
    return out




def plot_models_obs_hourly_mean(data=None, model_name=None, season=None,
                                out_filename=None,
                                out_dir=plots_output_path, sub_folder=None):
    '''
        Plots the models and observations average by hour
    
    '''
    W, H = 18, 8    # fig dimensions
    mean_hourly = data.groupby(data.index.time).mean()
    mean_hourly.index =  list(map(lambda t: t.hour, mean_hourly.index))
    mean_hourly.index.name = 'hour'

    fig, ax = plt.subplots(figsize=(W, H), frameon=True, constrained_layout=False)
    title = "{}: Average temperature for 3h time interval. All 16 {} models and its average.".format(*map(str.upper,[season, model_name])) 
    out = aux_plot_models_obs_hourly_mean(mean_hourly, ax, title=title)
    
    handles, labels = ax.get_legend_handles_labels()
    leg = ax.legend(handles[-2:], labels[-2:], frameon=False, loc=4, fontsize=14)
    
    
    if out_filename:
        if sub_folder:
            out_dir = os.path.join(out_dir, sub_folder)
        save_plot(plt, out_filename, output_dir = out_dir)
    
    return out



def plot_cmp_models_obs_hourly_mean(out_filename=None, title=u'title', out_dir=None, sub_folder=None):

    W, H = 18,28 # fig dimensions
    # composing (('cordex', 'DJF'), '..\\data\\cordex\\output'),  (('surfex', 'DJF'), '..\\data\\surfex\\output'),...,(('surfex', 'SON'), '..\\data\\surfex\\output')
    gen = zip(((mod,sea) for sea in seasons_dict.keys() for mod in ['cordex','surfex']),
        [cordex_output_path ,surfex_output_path]*4)

    fig, axs = plt.subplots(4,2, figsize=(W, H), frameon=True, constrained_layout=False, sharey=True)
    for ax,df_compound in zip (axs.flat, gen_cdx_sfx_ts_cmp_h_mean(models_seq=gen)):
        df, model, season = df_compound
        aux_plot_models_obs_hourly_mean(df, ax, title='{}: {}'.format(model, season))
    # get legend form last axe
    handles, labels = ax.get_legend_handles_labels()
    leg = fig.legend(handles, labels, ncol=4,frameon=False, loc=8, fontsize=14)
    
    fig.suptitle(title, fontsize=14)
    fig.subplots_adjust(top=0.95, bottom=0.1)                         # TRICK
    
    
    if out_filename:
        if sub_folder:
            out_dir = os.path.join(out_dir, sub_folder)
        save_plot(plt, out_filename, output_dir = out_dir)
        

  

In [26]:
def plot_cmp_metric_avg_h(metrics, data_ref, data_cmp, title_fig=None,
                    out_filename=None, out_dir=plots_output_path, sub_folder=None ):    
    '''
        OBS1:
            |avg(sigma)-1| or avg|sigma-1|?
            Here we adopt avg|sigma-1|
        OBS2:
            |Bias|.avg() or |Bias.avg()|?
            Adopted |Bias|.avg()
        OBS3:
            Computed the abs for all metrics: |Bias|,|Perkins|,|YK| and
            |Sigma-1|
    '''
    # fig dimensions
    W, H = 12, 12

    from itertools import cycle
    plot_id = cycle('abcdefghijklmnopqrstuvxwz')  


    fig, axs = plt.subplots(2,2, figsize=(W, H), frameon=True, constrained_layout=False)
    axs = axs.flat
    for metric,d_ref,d_cmp,ax in zip(metrics, data_ref, data_cmp, axs):
        # TODO -> function
        if metric == 'sigma':
            d_ref = d_ref-1
            d_cmp = d_cmp-1
        # ABS
        d_ref, d_cmp = d_ref.abs(), d_cmp.abs()
        
        data_x_min, data_x_max = d_ref.min(), d_ref.max()
        data_y_min, data_y_max = d_cmp.min(), d_cmp.max()
        x_min = min([data_x_min, data_x_max, data_y_min, data_y_max])
        x_max = max([data_x_min, data_x_max, data_y_min, data_y_max])
        if (x_max > 1) and (metric!='sigma'): x_max = np.ceil(x_max)                            # TODO BETTER

        
        data = pd.DataFrame()
        data['CDX'] = d_ref
        data['SFX'] = d_cmp
        plot = aux_plot_cmp_metric_avg_h(metric, ax,
                               'NOTHING',        # FIX IT TODO
                               data,
                               x_min, x_max,
                               plot_id,)

    # get legend form last axe
#     handles, labels = ax.get_legend_handles_labels()
#     leg = fig.legend(handles, labels, 
#                      ncol=3, mode='expand', 
#                      frameon=False, #loc=8,
#                      fontsize=14, markerscale=1, bbox_to_anchor=(0.5, 0.15))
    fig.suptitle('Season: {}'.format(title_fig), fontsize=14)
#     fig.tight_layout()
#     fig.subplots_adjust(top=0.95, bottom=0.1)                         # TRICK

    if out_filename:
        if sub_folder:
            out_dir = os.path.join(out_dir, sub_folder)
        save_plot(plt, out_filename, output_dir = out_dir)
        
        
      
        
# def plot_cmp_metric_avg_h_OLD(metric, data_ref, data_cmp, title_fig=None,
#                     out_filename=None, out_dir=plots_output_path, sub_folder=None ):
#     '''
#         OBS: Corrects for Sigma^-1
#     '''
#     # fig dimensions
#     W, H = 12, 24
    
    
#     if metric == 'sigma':
#         data_ref = 1/data_ref
#         data_cmp = 1/data_cmp

#     data_x_min, data_x_max = data_ref.min().min(), data_ref.max().max()
#     data_y_min, data_y_max = data_cmp.min().min(), data_cmp.max().max()
#     x_min = min([data_x_min, data_x_max, data_y_min, data_y_max])
#     x_max = max([data_x_min, data_x_max, data_y_min, data_y_max])
#     #x_min = np.floor(x_min)
#     if (x_max > 1) and (metric!='sigma'): x_max = np.ceil(x_max)                            # TODO BETTER

#     from itertools import cycle
#     plot_id = cycle('abcdefghijklmnopqrstuvxwz')  


# #     hours = range(0,22,3)
#     fig, axs = plt.subplots(2,2, figsize=(W, H), frameon=True, constrained_layout=False)
#     axs = axs.flat
#     for metric, ax in zip(hours, axs):
#         data = pd.DataFrame()
#         data['CDX'] = data_ref.loc[:,hour]
#         data['SFX'] = data_cmp.loc[:,hour]
#         plot = aux_plot_cmp_metric(metric, ax,
#                                hour,
#                                data,
#                                x_min, x_max,
#                                plot_id,)
#                                #param_dict={'zorder':1})

#     # get legend form last axe
#     handles, labels = ax.get_legend_handles_labels()
#     leg = fig.legend(handles, labels, 
#                      ncol=3, mode='expand', 
#                      frameon=False, loc=8,
#                      fontsize=14, markerscale=1)
#     fig.suptitle('Season: {}'.format(title_fig), fontsize=14)
# #     fig.tight_layout()
#     fig.subplots_adjust(top=0.95, bottom=0.1)                         # TRICK
    
#     if out_filename:
#         if sub_folder:
#             out_dir = os.path.join(out_dir, sub_folder)
#         save_plot(plt, out_filename, output_dir = out_dir)
        
        

## Assorted tools

In [27]:
def regional_markers(scale_markers):
    '''
        Function to invert the clustering of the markers to the regional name.
        scale_markers: dict made for the scale markers
    '''
    from itertools import chain
    del models_markers[18]
    del models_markers[19]
    regional_markers = map(lambda s: (s[0],'_'.join(s[1][0].split('_')[::-1])), models_markers.items())
    regional_markers = sorted(regional_markers, key=lambda i: i[1])
    regional_markers = zip(regional_markers,chain('d', '1'*2, '+'*4, '^', '*'*2, '2'*2, 'o'*5))
    return {pair[0]:(pair[1],marker)for pair, marker in regional_markers}

## Ensembles

#### Ensemble Alpha

In [28]:
def avg_diurnal_bias(bias_probe):
    avg_diurnal = pd.DataFrame(bias_probe, index=range(0,22,3))
    return pd.DataFrame(avg_diurnal[17]-avg_diurnal.iloc[:,:-1].mean(axis=1),
                       columns=[19])

def avg_diurnal_sigma(sigma_probe):
    avg_diurnal = pd.DataFrame(sigma_probe, index=range(0,22,3))
    return pd.DataFrame(avg_diurnal.iloc[:,:-1].mean(axis=1)/avg_diurnal[17],
                        columns=[19])

def avg_diurnal_perkins(perkins_probe):
    list_S = []
    for df_pdfs in perkins_probe:
        df_pdfs[ens_alpha_col] = df_pdfs.iloc[:,:-1].mean(axis=1)
        S = fn_models_obs(minimun, df_pdfs, [ens_alpha_col],
                          df_pdfs[temp_obs_col]).sum(axis=0)*100
        list_S.append(S.values)
    return pd.DataFrame(list_S, index=range(0,22,3), columns=[19])

def avg_diurnal_yk(yk_probe):
    list_yk = []
    for df_perc in yk_probe:
        df_perc[ens_alpha_col] = df_perc.iloc[:,:-1].mean(axis=1)
        list_yk.append(yk_perc(df_perc.loc[:,[temp_obs_col,ens_alpha_col]]))
    df_yk = pd.DataFrame(pd.concat(list_yk, axis=1).T)
    df_yk = df_yk[ens_alpha_col] - df_yk[temp_obs_col]
    return pd.DataFrame(df_yk.values, index=range(0,22,3), columns=[19])


def metrics_models_ensemblesALPHA(data_file, season, data_input_path, family_models,
                                 models_columns, obs_columns, ensemble_column,
                                 plot=False, sub_folder=None):
    '''
    Only works for Ensemble BETA: using all models' data points, with equal weights, as one single model.
    The problem is that it doesn't consider the model climate as a characteristic!
    '''
    # Load models and observations
    df_all_sea = load_csv2df(data_file, data_input_path, freq_index='3H')
    
    
    # Only the probes are used (probes give access to intermediate results)
    # BIAS
    df_bias_sea, bias_probe = metric_by_hour(bias, df_all_sea, models_columns, obs_columns)
    df_bias_ens_alpha = avg_diurnal_bias(bias_probe)
    df_bias_sea = pd.concat([df_bias_sea, df_bias_ens_alpha.T])
    

    # SIGMA
    # Calculate metrics for bare models
    df_sigma_sea, sigma_probe = metric_by_hour(sigma_score, df_all_sea, models_columns, obs_columns)
    df_sigma_ens_alpha = avg_diurnal_sigma(sigma_probe)
    df_sigma_sea = pd.concat([df_sigma_sea, df_sigma_ens_alpha.T])


    # PERKINS SKILL SCORE
    # Calculate metrics for bare models
    df_perkins_sea, perkins_probe = metric_by_hour(perkins_skill_score, df_all_sea, models_columns, obs_columns)
    df_perkins_ens_alpha = avg_diurnal_perkins(perkins_probe)
    df_perkins_sea = pd.concat([df_perkins_sea, df_perkins_ens_alpha.T])

    # YK SKEWNESS
    # Calculate metrics for bare models
    df_yk_sea, yk_probe = YK_skewness_by_hour(df_all_sea, models_columns, obs_columns)
    df_yk_ens_alpha = avg_diurnal_yk(yk_probe)
    df_yk_sea = pd.concat([df_yk_sea, df_yk_ens_alpha.T])

    if plot:
        plt_metrics_by_hour(df_bias_sea, '{}: Bias for 17 {} + Ensemble_ALPHA models by hour'.format(season, family_models),
                            filename='bias_{}_{}_ensALPHA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)
        plt_metrics_by_hour(df_sigma_sea, '{}: Sigma score for 17 {} + Ensemble_ALPHA models by hour'.format(season, family_models),
                            filename='sigma_{}_{}_ensALPHA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)
        plt_metrics_by_hour(df_perkins_sea, '{}: Perkins Skill score for 17 {} + Ensemble_ALPHA models by hour'.format(season, family_models),
                            filename='perkins_{}_{}_ensALPHA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)
        plt_metrics_by_hour(df_yk_sea, '{}: Yule-Kendall skewness for 17 {} + Ensemble_ALPHA models by hour'.format(season, family_models),
                            filename='yk_{}_{}_ensALPHA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)

    return df_bias_sea, df_sigma_sea, df_perkins_sea, df_yk_sea

#### Ensemble Beta

In [29]:
def metrics_models_ensemblesBETA(data_file, season, data_input_path, family_models,
                                 models_columns, obs_columns, ensemble_column,
                                 plot=False, sub_folder=None):
    '''
    Only works for Ensemble BETA: using all models' data points, with equal weights, as one single model.
    The problem is that it doesn't consider the model climate as a characteristic!
    
    Uses metric_by_hour() to compute metrics and probes (not used for ensemble BETA)
    Can plot using plt_metrics_by_hour()
    
    
    Returns: df_bias_sea, df_sigma_sea, df_perkins_sea, df_yk_sea
        values: metric
        cols: hours
        rows: models + ensemble
    
    '''
    # Load models and observations
    df_all_sea = load_csv2df(data_file, data_input_path, freq_index='3H')

    # Prepare df for ensemble calculations - BETA
    df_obs_sea, df_models_sea = df_all_sea[[obs_columns]], df_all_sea[models_columns]
    df_models_sea = df_models_sea.stack().to_frame(name=ensemble_column)
    df_models_sea.reset_index(level=1, drop=True, inplace=True)
    df_all_sea_ens = df_models_sea.join(df_obs_sea, how='left')
    df_all_sea_ens.head()

    # BIAS
    # Calculate metrics for bare models
    df_bias_sea, _ = metric_by_hour(bias, df_all_sea, models_columns, obs_columns)
    # Calculate bias for ensemble
    df_bias_sea_ens, _ = metric_by_hour(bias, df_all_sea_ens, [ensemble_column], obs_columns)
    # append
    df_bias_sea = df_bias_sea.append(df_bias_sea_ens)

    # SIGMA
    # Calculate metrics for bare models
    df_sigma_sea, _ = metric_by_hour(sigma_score, df_all_sea, models_columns, obs_columns)
    # Calculate bias for ensemble
    df_sigma_sea_ens, _ = metric_by_hour(sigma_score, df_all_sea_ens, [ensemble_column], obs_columns)
    # append
    df_sigma_sea = df_sigma_sea.append(df_sigma_sea_ens)

    # PERKINS SKILL SCORE
    # Calculate metrics for bare models
    df_perkins_sea, _ = metric_by_hour(perkins_skill_score, df_all_sea, models_columns, obs_columns)
    # Calculate bias for ensemble
    df_perkins_sea_ens, _ = metric_by_hour(perkins_skill_score, df_all_sea_ens, [ensemble_column], obs_columns)
    # append
    df_perkins_sea = df_perkins_sea.append(df_perkins_sea_ens)

    # YK SKEWNESS
    # Calculate metrics for bare models
    df_yk_sea, _ = YK_skewness_by_hour(df_all_sea, models_columns, obs_columns)
    # Calculate bias for ensemble
    df_yk_sea_ens, _ = YK_skewness_by_hour(df_all_sea_ens, [ensemble_column], obs_columns)
    # append
    df_yk_sea = df_yk_sea.append(df_yk_sea_ens)

    if plot:
        plt_metrics_by_hour(df_bias_sea, '{}: Bias for 17 {} + Ensemble_BETA models by hour'.format(season, family_models),
                            filename='bias_{}_{}_ensBETA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)
        plt_metrics_by_hour(df_sigma_sea, '{}: Sigma score for 17 {} + Ensemble_BETA models by hour'.format(season, family_models),
                            filename='sigma_{}_{}_ensBETA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)
        plt_metrics_by_hour(df_perkins_sea, '{}: Perkins Skill score for 17 {} + Ensemble_BETA models by hour'.format(season, family_models),
                            filename='perkins_{}_{}_ensBETA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)
        plt_metrics_by_hour(df_yk_sea, '{}: Yule-Kendall skewness for 17 {} + Ensemble_BETA models by hour'.format(season, family_models),
                            filename='yk_{}_{}_ensBETA'.format(season.lower(),family_models.lower()), sub_folder = sub_folder)

    return df_bias_sea, df_sigma_sea, df_perkins_sea, df_yk_sea

In [30]:
def aux_plot_cmp_metric_avg_h(metric, ax, hour, data, d_min, d_max, id_letter):#, param_dict):
    """
    A helper function to make a metric comparison graph

    Parameters
    ----------
    ax : Axes. The axes to draw to
    data : Dataframe. The data as a df with 2 cols
    d_min: the floor.min of all data points
    d_max: the ceil.max of all data points
    id_letter: letters to identify the subplots
    #param_dict : dict. Dictionary of kwargs to pass to ax.plot

    Returns
    -------
    out : list
        list of artists added
    """
    metric = metric.lower()
#     metrics_dict = {'bias': '|Bias|', 'sigma': r'|$\sigma_n^{-1}$|',    # TODO - mov to configurations file
#                     'perkins': '|S|', 's': '|S|', 'yk': '|YK|'}
    for idx,row in data.iterrows():
        out = ax.scatter(row[0], row[1], 
                         label=models_markers[idx][0], 
                         marker=models_markers[idx][1],
                         s=100,)
                        # **param_dict)
    if metric in ['bias', 'sigma', 'yk']:
        ax.fill([d_min, d_min, d_max, d_min], [d_min, d_max, d_max, d_min], fill=True, alpha=0.3, color='lightgray')
    elif metric in ['perkins', 's']: # only for perkins
        ax.fill([d_min, d_max, d_max, d_min], [d_min, d_min, d_max, d_min], fill=True, alpha=0.3, color='lightgray')
    else:
        print('Metric not allowed! Chose among: bias, perkins, sigma, yk')
    
    ax.set_xlabel(' '.join(('CORDEX {}'.format(metrics_symbols[metric]), r'T$_{2m}$ $^{o}$C')), fontsize=14)
    ax.set_ylabel(' '.join(('SFX {}'.format(metrics_symbols[metric]), r'T$_{2m}$ $^{o}$C')), fontsize=14)
    
    # outer limits
    dd_min, dd_max= d_min-0.05*d_max, d_max+0.05*d_max
    ax.set_xlim(dd_min, dd_max)
    ax.set_ylim(dd_min, dd_max)
    ax.text(-0.15, 0.95, '{})'.format(next(id_letter)),
            fontsize= 16, transform=ax.transAxes)
#     ax.text(0.5, 0.95, r'$={}h$'.format(),
#         horizontalalignment='center', fontsize=14, ha='center', transform=ax.transAxes)
    
    return out






        
        
        
        
        
def prepare_data_4x4(metrics_function, lst_data_files, input_path, model_family, models_columns, obs_columns, ensemble_column):
    from itertools import chain
    
    seasons_list = list(SEASONS)
    all_sea_all_met = []
    for file,sea in zip(lst_data_files, seasons_list):
#         lst_df = map(abs, metrics_function(file, sea, input_path, model_family, models_columns, obs_columns, ensemble_column))
        lst_df = metrics_function(file, sea, input_path, model_family, models_columns, obs_columns, ensemble_column)
        all_sea_all_met.append(lst_df)
    return chain(*zip(*all_sea_all_met))

        
        
def plot_cmp_metric_seasons_h_ensembles(data_ref, data_cmp, ensemble_column,
                                        title_fig=None, out_filename=None, out_dir=None, sub_folder=None ): 
    '''
        OBS:
            Computed the abs for all metrics: |Bias|,|Perkins|,|YK| and
            |Sigma-1|
    '''
    
    from itertools import cycle, chain
    
    # id each plot with a letter
    plot_id = cycle('abcdefghijklmnopqrstuvxwz')
    W, H = 18, 18
    # separate the 2 columns of points in x axis: (x-delta) for CORDEX; (x+delta) for SURFEX 
    delta = 0.45
    # produce the list: ['bias', 'bias', 'bias', 'bias', 'sigma',...,'sigma',..,'perkins', 'yk', 'yk', 'yk', 'yk']
    metrics = list(chain(*([metric]*4 for metric in METRIC_NAMES)))
    seasons_list_4 = list(SEASONS)*4


    #PLOTTING
    fig, axs = plt.subplots(4,4, figsize=(W, H), frameon=True, sharey='row') 
    axs = axs.flat
    for i,tup in enumerate(zip(seasons_list_4, metrics, axs, data_ref, data_cmp)):  # i: to implement cond. on the labels
#     for i,tup in enumerate(zip(seasons_list_4, metrics, axs, df_cdx, df_sfx)):  # i: to implement cond. on the labels        
        season,metric,ax,cdx,sfx = tup
        # cols: models; rows: hours -  to use pandas plot of a df
        cdx, sfx = cdx.copy().T, sfx.copy().T
        if metric == 'sigma':
            cdx = cdx-1
            sfx = sfx-1
        # ABS
        cdx, sfx = cdx.abs(), sfx.abs()
        
        # (x-delta)
        cdx = cdx.reset_index().rename({'index':'hour'}, axis=1)
        cdx['hour'] = cdx['hour']-delta
        # (x+delta)
        sfx = sfx.reset_index().rename({'index':'hour'}, axis=1)
        sfx['hour'] = sfx['hour']+delta
        # plot each column (model) except the 'hour' column
        for col in cdx.iloc[:,1:]:
            cdx.plot(x='hour',  y=col, kind='scatter', label='CORDEX', color = 'r', marker='.', ax=ax)
            sfx.plot(x='hour',  y=col, kind='scatter', label='SURFEX', color='c', marker= '.', ax=ax) 
        # plot the ensembles
        cdx.plot(x='hour',  y=ensemble_column, kind='scatter', label='Ensemble - CORDEX', color='k', marker='$C$', s=60, ax=ax)
        sfx.plot(x='hour',  y=ensemble_column, kind='scatter', label='Ensemble - SURFEX', color='k',  marker='$S$', s=60, ax=ax) 
        ax.set_xticks(list(range(0,22,3)))    

        # ylabel only for the left column
        if i in [0,4,8,12]:
            ax.set_ylabel(' '.join(('{}'.format(metrics_symbols[metric]), r'T$_{2m}$ $^{o}$C')), fontsize=14)
        else:
            ax.set_ylabel(None)
        # xlabel only fo thr bottom row
        if i >= 12:
            ax.set_xlabel('hour')
        else:
            ax.set_xlabel(None)
        # letter id for all plots
        ax.text(-0.15, 1.05, '{})'.format(next(plot_id)),
                fontsize= 14, transform=ax.transAxes)
        # season name for top row
        if i in [0,1,2,3]:
            ax.text(0.5, 1.05, r'{}'.format(season),
                horizontalalignment='center', fontsize=14, ha='center', transform=ax.transAxes)
        else:
            pass
        ax.get_legend().remove()                       # pandas by default draws the legend   

    # get legend form last axe and make 2 separted legends so the first one has the scatterpoints=4
    handles, labels = ax.get_legend_handles_labels()
    # make the legend for CORDEX and SURFEX
    _ = fig.legend(handles[-4:-2], labels[-4:-2],  ncol=2, frameon=True, loc=8, fontsize=14, markerscale=2, # scales marker relative to plot
                   bbox_to_anchor=(0.35, 0.05),           # controls the position
                   bbox_transform=plt.gcf().transFigure,  # coords refrenced to the figure
                   scatterpoints=4)
    # make the second legend for the ensembles
    _ = fig.legend(handles[-2:], labels[-2:], ncol=2, frameon=True, loc=8, fontsize=14,
                         bbox_to_anchor=(0.65, 0.05),
                         bbox_transform=plt.gcf().transFigure)
    if not title_fig:
        title_fig = r'T$_{2m}$ Error metrics for CORDEX and SURFEX models and its respective Ensembles.'
    fig.suptitle(title_fig, fontsize=14)
    fig.subplots_adjust(top=0.93, bottom=0.1)
#     fig.tight_layout() 
    
    if out_filename:
        if sub_folder:
            out_dir = os.path.join(out_dir, sub_folder)
        save_plot(plt, out_filename, output_dir = out_dir)
        
        
 

In [31]:
def boxplot_cmp_metric_seasons_h_ensembles(data_ref, data_cmp, ensemble_column,
                                        title_fig=None, out_filename=None, out_dir=None, sub_folder=None ): 
    '''
        OBS:
            Computed the abs for all metrics: |Bias|,|Perkins|,|YK| and
            |Sigma-1|
    '''
    
    
    
    from itertools import cycle, chain
    from matplotlib.patches import Polygon
    
    def separate_ens_data(data_arr):
        new_data, ens_data = [], []
        for item in data_arr:
            ens_data.append(item[-1])
            item = np.delete(item, -1, 0)
            new_data.append(item)
        return new_data, np.array(ens_data) 
    
    # id each plot with a letter
    plot_id = cycle('abcdefghijklmnopqrstuvxwz')
    W, H = 18, 18
    # produce the list: ['bias', 'bias', 'bias', 'bias', 'sigma',...,'sigma',..,'perkins', 'yk', 'yk', 'yk', 'yk']
    metrics = list(chain(*([metric]*4 for metric in METRIC_NAMES)))
    seasons_list_4 = list(SEASONS)*4
    color_cdx, color_sfx = 'c','y'#'lightcyan', 'lightsalmon'

    #PLOTTING
    fig, axs = plt.subplots(4,4, figsize=(W, H), frameon=True, sharey='row') 
    axs = axs.flat
    for i,tup in enumerate(zip(seasons_list_4, metrics, axs, data_ref, data_cmp)):  # i: to implement cond. on the labels
        season,metric,ax,cdx,sfx = tup
        # cols: models; rows: hours -  to use pandas plot of a df
        cdx, sfx = cdx.copy(), sfx.copy()
        
        if metric == 'sigma':
            cdx = cdx - 1
            sfx = sfx - 1
        # ABS
        cdx, sfx = cdx.abs(), sfx.abs()
        
        data = list(chain(*zip(cdx.values.T, sfx.values.T)))
        data, ensemble = separate_ens_data(data)
#         print(season,metric, data, ensemble)
        
        bp = ax.boxplot(data, notch=0, sym='+', vert=1, whis=1.5)
        
        plt.setp(bp['boxes'], color='black')
        plt.setp(bp['whiskers'], color='black')
        plt.setp(bp['medians'], color='black')
        plt.setp(bp['fliers'], color='black', marker='.')
        
        # the first xtick is 0 we have to put a dummy number there
        head_0 = np.ma.masked_where(1, [0], copy=True)
        cdx_mask = np.ma.masked_where(np.arange(0,16)%2, ensemble, copy=True)
        cdx_mask = np.ma.append(head_0,cdx_mask)
        sfx_mask = np.ma.masked_where(~np.arange(0,16)%2, ensemble, copy=True)
        sfx_mask = np.ma.append(head_0,sfx_mask)
        ax.plot(cdx_mask, color='k', marker='$C$', label='Ensemble - CORDEX')
        ax.plot(sfx_mask, color='k', marker='$S$', label='Ensemble - SURFEX')

        # Add a horizontal grid to the plot, but make it very light in color
        # so we can use it for reading data values but not be distracting
        ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
                       alpha=0.5)

        # Now fill the boxes with desired colors
        box_colors = [color_cdx, color_sfx]
        num_boxes = len(data)
        medians = np.empty(num_boxes)
        for j in range(num_boxes):
            box = bp['boxes'][j]
            box_x, box_y = [], []
            # dict_keys(['whiskers', 'caps', 'boxes', 'medians', 'fliers', 'means'])
            for m in range(5): 
                box_x.append(box.get_xdata()[m])
                box_y.append(box.get_ydata()[m])
            box_coords = np.column_stack([box_x, box_y])
            # Alternate between color_cdx and color_sfx
            ax.add_patch(Polygon(box_coords, facecolor=box_colors[j % 2]))

        # delete tick labels
        ax.set_xticklabels(['' for i in range(17)])
        # re-make the tick labels
        pos = np.arange(num_boxes) + 1
        labels = np.repeat(range(0,22,3), 2)
        for tick, label in zip(range(num_boxes), ax.get_xticklabels()):
            k = tick % 2
            ax.text(pos[tick]+0.5, -0.05, labels[tick],
                 transform=ax.get_xaxis_transform(),
                 horizontalalignment='center',
                 color='w' if k else 'k')

        # ylabel only for the left column
        if i in [0,4,8,12]:
            ax.set_ylabel(' '.join(('{}'.format(metrics_symbols[metric]), r'T$_{2m}$ $^{o}$C')), fontsize=14)
        else:
            ax.set_ylabel(None)
        # xlabel only fo thr bottom row
        if i >= 12:
            ax.set_xlabel('hour')
        else:
            ax.set_xlabel(None)
        # letter id for all plots
        ax.text(-0.15, 1.05, '{})'.format(next(plot_id)),
                fontsize= 14, transform=ax.transAxes)
        # season name for top row
        if i in [0,1,2,3]:
            ax.text(0.5, 1.05, r'{}'.format(season),
                horizontalalignment='center', fontsize=14, ha='center', transform=ax.transAxes)
        else:
            pass

    if not title_fig:
        title_fig = r'T$_{2m}$ Error metrics for CORDEX and SURFEX models and its respective Ensembles.'
    fig.suptitle(title_fig, fontsize=14)
    fig.subplots_adjust(top=0.93, bottom=0.05)
    

    # Finally, add a basic legend
    fig.text(0.30, 0.015, 'CORDEX',
             backgroundcolor=box_colors[0], color='black')#, size='x-small')
    fig.text(0.37, 0.015, 'SURFEX',
             backgroundcolor=box_colors[1], color='black')#, size='x-small')
    fig.text(0.45, 0.015, 'C Esemble-CORDEX', color='black')#, size='x-small')
    fig.text(0.55, 0.015, 'S Ensemble-SURFEX', color='black')#, size='x-small')
    
    if out_filename:
        if sub_folder:
            out_dir = os.path.join(out_dir, sub_folder)
        save_plot(plt, out_filename, output_dir = out_dir)